# Renko & MACD

In [1]:
import pandas_datareader.data as pdr
import pandas as pd
import numpy as np
import datetime
import copy
import sys
import warnings

from stocktrends import Renko
from itertools import product

warnings.simplefilter(action='ignore')
sys.path.append("../")

from data.make_dataset import get_stock_intraday
from technical_indicators import get_atr, get_obv, get_renko, get_slopes, get_macd
from kpi import get_sharpe, get_cagr, get_max_drawdown, get_calmar

/anaconda3/envs/fx/lib/python3.8/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
tickers = ["CSCO","VZ","IBM","QCOM","LYFT"]
ohlc_intraday = get_stock_intraday("alpha_vantage_key.txt", tickers)

CSCO
VZ
IBM
QCOM
LYFT


In [3]:
ohlc_renko = {}
dfs = copy.deepcopy(ohlc_intraday)

for ticker in tickers:
    print("Merging for: ", ticker)
    #dfs[ticker].reset_index(inplace=True)
    renko = get_renko(dfs[ticker])
    tmp_df = dfs[ticker].merge(renko[['date', 'renko_bar']], how='outer', on='date')
    tmp_df['renko_bar'].fillna(method='ffill', inplace=True)
    tmp_df['macd'], tmp_df['macd_sig'] = get_macd(tmp_df, 'Adj Close')
    tmp_df['macd_slope'] = get_slopes(tmp_df['macd'], degree=True)
    tmp_df['macd_sig_slope'] = get_slopes(tmp_df['macd_sig'], degree=True)
    ohlc_renko[ticker] = tmp_df
    del tmp_df

Merging for:  CSCO
Merging for:  VZ
Merging for:  IBM
Merging for:  QCOM
Merging for:  LYFT


In [4]:
ticker_signal = {ticker: None for ticker in tickers}
ticker_return = {ticker: [] for ticker in tickers}

In [5]:
def buy_signal(df, i, renko_nb=2):
    signal = df['renko_bar'][i] >= renko_nb and df['macd'][i] > df['macd_sig'][i] and\
        df['macd_slope'][i] >df['macd_sig_slope'][i]
    return signal


def sell_signal(df, i, renko_nb=2):
    signal = df['renko_bar'][i] <= -renko_nb and df['macd'][i] < df['macd_sig'][i] and\
        df['macd_slope'][i] < df['macd_sig_slope'][i]
    return signal


def exit_signal(df, i, position):
    if position == -1:
        signal = df['macd'][i] > df['macd_sig'][i] and df['macd_slope'][i] > df['macd_sig_slope'][i]
    elif position == 1:
        signal = df['macd'][i] < df['macd_sig'][i] and df['macd_slope'][i] < df['macd_sig_slope'][i]
    return signal

In [6]:
def backtest(ohlc_renko, renko_nb, verbose=True):
    for ticker in tickers:
        if verbose:
            print(f"Calculating daily returns for: {ticker}")
        try:
            del df
        except NameError:
            pass
        finally: 
            df = ohlc_renko[ticker].reset_index()
        for i, _ in df.iterrows():
            if ticker_signal[ticker] is None or ticker_signal[ticker] == 0:
                ticker_return[ticker].append(0)
                if i > 0 and buy_signal(df, i, renko_nb):
                    ticker_signal[ticker] = 1
                elif i > 0 and sell_signal(df, i, renko_nb):
                    ticker_signal[ticker] = -1
            
            elif ticker_signal[ticker] == 1:
                ticker_return[ticker].append((df['Adj Close'][i]/df['Adj Close'][i-1])-1)
                if sell_signal(df, i, renko_nb):
                    ticker_signal[ticker] = -1
                elif exit_signal(df, i, position=1):
                    ticker_signal[ticker] = 0
                    
            elif ticker_signal[ticker] == -1:
                ticker_return[ticker].append((df['Adj Close'][i]/df['Adj Close'][i-1])-1)
                if buy_signal(df, i, renko_nb):
                    ticker_signal[ticker] = 1
                elif exit_signal(df, i, position=-1):
                    ticker_signal[ticker] = 0
        ohlc_renko[ticker]['returns'] = ticker_return[ticker]
    returns_df = pd.DataFrame({ticker: ohlc_renko[ticker]['returns'] for ticker in tickers})
    returns_df['total_returns'] = returns_df.sum(axis=1)
    return returns_df


In [7]:
results = backtest(ohlc_renko, renko_nb=2, verbose=True)

Calculating daily returns for: CSCO
Calculating daily returns for: VZ
Calculating daily returns for: IBM
Calculating daily returns for: QCOM
Calculating daily returns for: LYFT


In [8]:
cagr = {}
sharpe = {}
max_dd = {}
calmar = {}
for col in results:
    print("Calculating KPIs for: ", col)
    cagr[col] = get_cagr(results, col, 252*78)
    sharpe[col] = get_sharpe(results, col, 0.0225, 252*78)
    max_dd[col] = get_max_drawdown(results, col)
    calmar[col] = get_calmar(results, col, annualized_const=0.0225)

Calculating KPIs for:  CSCO
Calculating KPIs for:  VZ
Calculating KPIs for:  IBM
Calculating KPIs for:  QCOM
Calculating KPIs for:  LYFT
Calculating KPIs for:  total_returns


In [11]:
kpi_df = pd.DataFrame([cagr, sharpe, max_dd, calmar], index=['cagr', 'sharpe', 'max_dd', 'calmar']).T

In [12]:
kpi_df.sort_values('sharpe', ascending=False)

,cagr,sharpe,max_dd,calmar
IBM,4.485,15.446,0.038,0.0
total_returns,12.934,10.641,0.172,0.0
LYFT,6.062,9.379,0.103,0.0
QCOM,-0.177,-0.684,0.056,-0.0
CSCO,-0.157,-1.076,0.058,-0.0
VZ,-0.242,-1.734,0.041,-0.0
